In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# MCP Toolbox on Vertex AI Agent Engine with custom installation scripts

## Overview

Vertex AI Agent Engine provides **support for custom installation and startup scripts**. This means you can now run shell scripts you need during your agent's build and startup process. You can install custom binaries, compile code from source, or set up any other specialized dependency your agent requires.

Most importantly, this feature means unlocks a new pattern for deploying agents with complex, standardized tools. In fact, you can now **deploy agents using the Model Context Protocol (MCP) on Vertex AI Agent Engine**.

This guide shows you how to build, test, and deploy a custom Reddit agent that uses an MCP server, demonstrating this new pattern for creating custom agents on Vertex AI Agent Engine.

**What You'll Learn**

  * How to set up your local environment for agent development.
  * How to run MCP Toolbox locally with Python, BigQuery, and ADK.
  * How to build and test an agent locally using the Google Agent Development Kit (ADK).
  * How to use the "bring your own installation and startup script" feature to deploy the agent w/ MCP toolbox to Vertex AI Agent Engine.


## Get started

### Install Google Gen AI SDK and other required packages

Run the following command to install the Vertex AI SDK with the necessary extras for Agent Engine and the ADK.

In [17]:
%pip install "google-cloud-aiplatform[agent_engines,adk]" "aiofiles" "cryptography==3.4.8"--upgrade --quiet

ERROR: Invalid requirement: 'cryptography==3.4.8--upgrade': Expected end or semicolon (after version specifier)
    cryptography==3.4.8--upgrade
                ~~~~~~~^


**Important:** After the installation completes, you must restart your Colab or notebook kernel for the new packages to be recognized.


### Authenticate your notebook environment (Colab only)

Next, authenticate your account and initialize the Vertex AI SDK. This allows your environment to interact with your Google Cloud project.


In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).


In [ ]:
# Use the environment variable if the user doesn't provide Project ID.
import os

PROJECT_ID = ""  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "global")

LOCATION = "us-central1"

BUCKET_NAME = ""  # @param {type: "string", placeholder: "[your-bucket-name]", isTemplate: true}
BUCKET_URI = f"gs://{BUCKET_NAME}"

#! gsutil mb -l {LOCATION} -p {PROJECT_ID} {BUCKET_URI}

# Set environment variables required for ADK
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "TRUE"
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
os.environ["GOOGLE_CLOUD_LOCATION"] = LOCATION

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

## Import required libraries

Import the necessary libraries from the ADK, Vertex AI SDK, and Python's standard library.


In [2]:
import logging

logging.getLogger("google_adk").setLevel(logging.CRITICAL)
logging.getLogger("asyncio").setLevel(logging.CRITICAL)
import os
import uuid
from typing import Any, Iterator, Optional

import aiofiles
from google.adk.agents import LlmAgent
from google.adk.artifacts import InMemoryArtifactService
from google.adk.memory import InMemoryMemoryService
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools.mcp_tool import StdioConnectionParams
from google.adk.tools.mcp_tool.mcp_toolset import (MCPToolset,
                                                   StdioServerParameters)
from google.genai import types
from vertexai import agent_engines
from vertexai.agent_engines import AgentEngine
from vertexai.preview.reasoning_engines import AdkApp
from toolbox_core import ToolboxSyncClient


### Helper function

This tutorial uses a helper function, `chat_loop`, to provide a consistent interactive chat interface for testing the agent in both local and deployed environments. This function abstracts the differences between interacting with a local ADK Runner and a remote AgentEngine instance.


In [3]:
def chat_loop(
    app, user_id: Optional[str] = None, session_id: Optional[str] = None
) -> None:
    """Interactive chat loop for AI applications."""

    # Simple setup
    user_id = user_id or f"u_{uuid.uuid4().hex[:8]}"

    # Handle session based on app type
    if isinstance(app, (AdkApp, AgentEngine)):
        # Only create session if session_id is not provided
        if not session_id:
            session = app.create_session(user_id=user_id)
            # Handle both dict and object session responses
            if isinstance(session, dict):
                session_id = session["id"]
            else:
                session_id = session.id

        def query_fn(msg: str):
            return app.stream_query(user_id=user_id, session_id=session_id, message=msg)

    elif isinstance(app, Runner):
        session_id = session_id or f"s_{uuid.uuid4().hex[:8]}"

        def query_fn(msg: str):
            return app.run(
                user_id=user_id,
                session_id=session_id,
                new_message=types.Content(role="user", parts=[types.Part(text=msg)]),
            )

    else:
        raise TypeError(
            f"Unsupported app type: {type(app)}. Expected AdkApp, AgentEngine, or Runner."
        )

    _print_startup_info(user_id, session_id)

    # Main loop
    while True:
        try:
            user_input = input("\nYou: ").strip()
            if not user_input or user_input.lower() in {"quit", "exit", "bye"}:
                break

            print("\nAssistant: ", end="", flush=True)

            response = _get_response_text(query_fn(user_input))
            print(response or "(No response generated)")

        except (KeyboardInterrupt, EOFError):
            print("\n\n🛑 Chat interrupted.")
            break
        except Exception as e:
            print(f"\n❌ Error: {e}")
            continue

    print("\n👋 Goodbye!")


def _print_startup_info(user_id: str, session_id: str) -> None:
    """Print startup information."""
    print("\n🚀 Starting chat...")
    print(f"👤 User ID: {user_id}")
    print(f"📁 Session ID: {session_id}")
    print("💬 Type 'exit' or 'quit' to end.")
    print("-" * 50)


def _get_response_text(events: Iterator[Any]) -> str:
    """Extract response text from event stream."""
    responses = []

    for event in events:
        # Handle dict-like events (AgentEngine format)
        if isinstance(event, dict):
            text = _extract_from_dict_event(event)
        # Handle object-like events
        else:
            text = _extract_from_object_event(event)

        if text:
            responses.append(text)
            # Print streaming text in real-time
            print(text, end="", flush=True)

    return "".join(responses)


def _extract_from_dict_event(event: dict) -> Optional[str]:
    """Extract text from dictionary-style events (AgentEngine format)."""
    # Handle AgentEngine response format
    if "parts" in event and "role" in event:
        # Only extract text from model responses, skip function calls/responses
        if event.get("role") == "model":
            parts = event.get("parts", [])
            text_parts = []

            for part in parts:
                # Extract text content, skip function calls
                if isinstance(part, dict) and "text" in part:
                    text_parts.append(part["text"])

            return "".join(text_parts) if text_parts else None
        return None

    # Handle other dict formats
    content = event.get("content", {})
    if isinstance(content, str):
        return content

    parts = content.get("parts", [])
    if not parts:
        return None

    text_parts = []
    for part in parts:
        if isinstance(part, dict) and "text" in part:
            text_parts.append(part["text"])

    return "".join(text_parts) if text_parts else None


def _extract_from_object_event(event: Any) -> Optional[str]:
    """Extract text from object-style events."""
    # Handle string content directly
    content = getattr(event, "content", None)
    if isinstance(content, str):
        return content

    # Handle content with parts
    if content and hasattr(content, "parts"):
        text_parts = []
        for part in content.parts:
            if hasattr(part, "text") and part.text:
                text_parts.append(part.text)
        return "".join(text_parts) if text_parts else None

    # Handle direct text attribute
    return getattr(event, "text", None)

## Building the Agent Locally

Before deployment, the agent is built and tested in the local environment. This allows for rapid iteration and debugging.


### Prepare the installation script

The agent uses an external tool, `mcp-reddit`, which must be installed in the runtime environment. A shell script will be created to handle this installation. Vertex AI Agent Engine will execute this script during the deployment process.

First, create a local directory to store the script.

Then, the script performs the following actions:
- Updates the package index and installs curl.
- Installs uv, a fast Python package manager, to accelerate dependency installation.
- Adds uv to the system's PATH.
- Uses uv to install the mcp-reddit package directly from its Git repository.


In [4]:
!mkdir -p installation_scripts

In [6]:
install_local_mcp_file = """
#!/bin/bash
# Exit immediately if a command exits with a non-zero status.
set -e

echo "Installing MCP Toolbox Server"

# Install uv (a fast Python package manager)
apt-get update
apt-get install -y curl

echo "Installing MCP Toolbox"

export OS="linux/amd64" # one of linux/amd64, darwin/arm64, darwin/amd64, or windows/amd64
curl -O https://storage.googleapis.com/genai-toolbox/v0.12.0/$OS/toolbox

pwd
ls -ltr

chmod 755 toolbox

# Install the MCP Toolbox using the command from its documentation

echo "Installing uv"

curl -LsSf https://astral.sh/uv/install.sh | env UV_INSTALL_DIR="/usr/local/bin" sh

# Add uv to PATH for current session
export PATH="$HOME/.local/bin:$PATH"

# Install the MCP Toolbox using the command from its documentation
echo "Installing MCP Toolbox using uv..."
uv pip install toolbox-core --system
echo "MCP Toolbox Server installation complete."
"""

with open("installation_scripts/install_local_mcp.sh", "w") as f:
    f.write(install_local_mcp_file)
f.close()

In [ ]:
!mkdir -p startup_scripts

In [7]:
init_uv_file = """
#!/bin/bash
# Exit immediately if a command exits with a non-zero status.
set -e

echo "Starting MCP Toolbox Server"

./toolbox --tools-file "tools.yaml"

"""

with open("startup_scripts/init_uv.sh", "w") as f:
    f.write(init_uv_file)
f.close()

In [ ]:
!chmod +x installation_scripts/install_local_mcp.sh && ./installation_scripts/install_local_mcp.sh

### Defining the Agent

Here, we define the `LlmAgent` using ADK. We provide a detailed prompt and connect it to our `MCPToolset`, pointing it to the runner script. We define our agent's using a factory function. The main test loop will then call this function to create an agent instance, providing it with the necessary async log file handle at runtime.

> **Notice**: To solve the `fileno` error in Colab, we must redirect the tool's error stream to a file that supports asynchronous operations. We will use the aiofiles library for this. This requires our test to run inside an async function.

In [9]:
def create_agent(errlog):
    root_agent = LlmAgent(
        model="gemini-2.5-flash",
        name="reddit_assistant_agent",
        instruction="Help the user fetch reddit info.",
        tools=[
            MCPToolset(
                connection_params=StdioConnectionParams(
                    server_params=StdioServerParameters(
                        command="mcp-reddit",
                    ),
                ),
                errlog=errlog,  # Required only in colab environment
            )
        ],
    )
    return root_agent

In [10]:
def create_agent(errlog):

    toolbox_client = ToolboxSyncClient("http://127.0.0.1:5000")
    agent_toolset = toolbox_client.load_toolset("my-toolset")

    # --- Define the Agent's Prompt ---
    prompt = """
      You're a helpful hotel assistant. You handle hotel searching, booking and
      cancellations. When the user searches for a hotel, mention it's name, id,
      location and price tier. Always mention hotel ids while performing any
      searches. This is very important for any operations. For any bookings or
      cancellations, please provide the appropriate confirmation. Be sure to
      update checkin or checkout dates if mentioned by the user.
      Don't ask for confirmations from the user.
    """

    root_agent = LlmAgent(
        model='gemini-2.5-flash',
        name='hotel_agent',
        description='A helpful AI assistant that can search and book hotels.',
        instruction=prompt,
        tools=agent_toolset, # Pass the loaded toolset
    )
    return root_agent

### Test the agent locally

Test the agent on the local machine to verify its functionality before deploying to the cloud.

The local test setup involves:
- Instantiating in-memory versions of the SessionService, MemoryService, and ArtifactService for lightweight local testing.
- Creating an async file handle for the tool's error log.
- Initializing the ADK Runner, which orchestrates the interaction between the user and the agent.
- Calling the chat_loop helper to start an interactive conversation with the agent.


In [11]:
async def test_agent_locally():

    # Create the session service
    session_service = InMemorySessionService()
    memory_service = InMemoryMemoryService()  # Add this line
    artifact_service = InMemoryArtifactService() # Add this line

    # Create the specific session where the conversation will happen
    user_id = "runner_user_01"
    session = await session_service.create_session(
        app_name="MyRunnerApp", user_id=user_id
    )

    # Initialize the Runner with the correct session service
    errlog = await aiofiles.open(
        "error.log", "w+"
    )  # Required only in colab environment.
    root_agent = create_agent(errlog)

    runner = Runner(
        agent=root_agent,
        app_name="MyRunnerApp",
        session_service=session_service,
        memory_service=memory_service,
        artifact_service=artifact_service,
    )

    try:
        chat_loop(runner, user_id, session.id)
    finally:
        # Ensure the log file is always closed
        await errlog.close()

Follow Step 1 in https://googleapis.github.io/genai-toolbox/samples/bigquery/local_quickstart/ to set up your BigQuery Dataset and Table

In [12]:
await test_agent_locally()


🚀 Starting chat...
👤 User ID: runner_user_01
📁 Session ID: bae6687b-a4a6-468d-909a-7d54ea6022c4
💬 Type 'exit' or 'quit' to end.
--------------------------------------------------

You: hi

Assistant: Hello! I'm a helpful hotel assistant. I can search, book, and cancel hotel reservations for you. How may I help you today?
Hello! I'm a helpful hotel assistant. I can search, book, and cancel hotel reservations for you. How may I help you today?


You: book a hotel in zurich

Assistant: 

Here are the hotels I found in Zurich:

*   **Courtyard Zurich** (ID: 9) - Location: Zurich, Price Tier: Upscale
*   **Marriott Zurich** (ID: 2) - Location: Zurich, Price Tier: Upscale
*   **Sheraton Zurich** (ID: 7) - Location: Zurich, Price Tier: Upper Upscale

Which hotel would you like to book? Please provide the Hotel ID.Here are the hotels I found in Zurich:

*   **Courtyard Zurich** (ID: 9) - Location: Zurich, Price Tier: Upscale
*   **Marriott Zurich** (ID: 2) - Location: Zurich, Price Tier: Upscale
*   **Sheraton Zurich** (ID: 7) - Location: Zurich, Price Tier: Upper Upscale

Which hotel would you like to book? Please provide the Hotel ID.

You: quit

👋 Goodbye!


## Deploying to Vertex AI Agent Engine

With our agent tested and working locally, it's time to deploy our MCP-enabled agent.


### Create the agent module

Since ADK 1.0.0, the `MCPToolset` contains non-serializable (non-pickleable) state, such as thread locks.

To deploy an agent with such a toolset, it must be wrapped in a `ModuleAgent`. This approach involves defining the agent in a separate Python file (`root_agent.py`) and referencing it by the module and variable name during deployment.


This agent module file defines the full application, including service builders for `VertexAiSessionService`, which are the cloud-based counterparts to the in-memory services used for local testing. The agent itself is wrapped in an AdkApp object.

In [13]:
root_agent_file = f"""
import os
from google.adk.agents import LlmAgent
from google.adk.tools.mcp_tool.mcp_toolset import MCPToolset, StdioServerParameters
from google.adk.tools.mcp_tool import StdioConnectionParams
from vertexai.preview.reasoning_engines import AdkApp
from toolbox_core import ToolboxSyncClient

# Set variable
PROJECT_ID=os.getenv('PROJECT_ID', '{PROJECT_ID}')
LOCATION=os.getenv('LOCATION', '{LOCATION}')
BUCKET_NAME=os.getenv('BUCKET_NAME', '{BUCKET_NAME}')

# Define session builder
def session_service_builder():
  # This is needed to ensure InitGoogle and AdkApp setup is called first.
  from google.adk.sessions import VertexAiSessionService
  return VertexAiSessionService(project=PROJECT_ID, location=LOCATION)

toolbox_client = ToolboxSyncClient("http://127.0.0.1:5000")
agent_toolset = toolbox_client.load_toolset("my-toolset")

agent_app = AdkApp(
    agent=LlmAgent(
        model='gemini-2.5-flash',
        name='hotel_agent',
        description='A helpful AI assistant that can search and book hotels.',
        instruction='You are a helpful hotel assistant. You handle hotel searching, booking and cancellations. When the user searches for a hotel, mention its name, id, location and price tier. Always mention hotel ids while performing any searches. This is very important for any operations. For any bookings or cancellations, please provide the appropriate confirmation. Be sure to update checkin or checkout dates if mentioned by the user. Do not ask for confirmations from the user.',
        tools=agent_toolset, # Pass the loaded toolset
    ),
    session_service_builder=session_service_builder
)
"""

with open("root_agent.py", "w") as f:
    f.write(root_agent_file)
f.close()

### Deploy the agent

The agent_engines.create() function is used to deploy the agent. The configuration specifies several key parameters:

- `agent_engine`: An `agent_engines.ModuleAgent` is used, pointing to the `root_agent.py` file (module_name) and the `agent_app` variable within it (agent_name).

- `requirements`: A list of standard Python package dependencies.

- `extra_packages`: A list of local files to be included in the build. This must include the agent module file (`root_agent.py`) and the custom installation script.

- `env_vars`: A dictionary of environment variables to be set in the deployed container, used here to securely pass the Reddit API credentials.

- `build_options`: This dictionary specifies custom build-time operations. The installation key is used to provide a list of scripts that will be executed before the application server starts. This is where the custom installation script is specified.


In [14]:
remote_app = agent_engines.create(
    display_name="bigquery_toolbox_agent",
    description="A bigquery toolbox agent with MCP",
    agent_engine=agent_engines.ModuleAgent(
        module_name="root_agent",
        agent_name="agent_app",
        register_operations={
            "": ["get_session", "list_sessions", "create_session", "delete_session"],
            "async": [
                "async_get_session",
                "async_list_sessions",
                "async_create_session",
                "async_delete_session",
            ],
            "stream": ["stream_query", "streaming_agent_run_with_events"],
            "async_stream": ["async_stream_query"],
        },
    ),
    requirements=["google-cloud-aiplatform[agent_engines,adk]>=1.101.0","toolbox-core"],
    extra_packages=[
        "root_agent.py",
        "startup_scripts/init_uv.sh",
        "installation_scripts/install_local_mcp.sh",
        "tools.yaml",
    ],
    env_vars={
        "PROJECT_ID": PROJECT_ID,
        "LOCATION": LOCATION,
    },
    build_options={
        "installation": [
            "installation_scripts/install_local_mcp.sh",
        ],
    },
)

INFO:vertexai.agent_engines:Identified the following requirements: {'google-cloud-aiplatform': '1.122.0', 'cloudpickle': '3.1.1', 'pydantic': '2.12.3'}
INFO:vertexai.agent_engines:The following requirements are appended: {'pydantic==2.12.3', 'cloudpickle==3.1.1'}
INFO:vertexai.agent_engines:The final list of requirements: ['google-cloud-aiplatform[agent_engines,adk]>=1.101.0', 'toolbox-core', 'pydantic==2.12.3', 'cloudpickle==3.1.1']
INFO:vertexai.agent_engines:Using bucket zyagent-staging
INFO:vertexai.agent_engines:Wrote to gs://zyagent-staging/agent_engine/agent_engine.pkl
INFO:vertexai.agent_engines:Writing to gs://zyagent-staging/agent_engine/requirements.txt
INFO:vertexai.agent_engines:Creating in-memory tarfile of extra_packages
INFO:vertexai.agent_engines:Writing to gs://zyagent-staging/agent_engine/dependencies.tar.gz
/usr/local/lib/python3.11/dist-packages/vertexai/preview/reasoning_engines/templates/adk.py:587: UserWarning: [EXPERIMENTAL] InMemoryCredentialService: This feat

After deployment, the chat_loop function can be used again to interact with the now remotely-hosted agent.

In [15]:
chat_loop(remote_app)


🚀 Starting chat...
👤 User ID: u_c61e89e6
📁 Session ID: 1273476907278532608
💬 Type 'exit' or 'quit' to end.
--------------------------------------------------

Assistant: Hello! I'm a hotel assistant. I can help you with searching, booking, and canceling hotels. How may I help you today?
Hello! I'm a hotel assistant. I can help you with searching, booking, and canceling hotels. How may I help you today?


Assistant: Great! Do you have a specific hotel in mind, or are you looking for suggestions in a particular location? I'll need the hotel's name or ID to book it for you.Great! Do you have a specific hotel in mind, or are you looking for suggestions in a particular location? I'll need the hotel's name or ID to book it for you.

Assistant: I couldn't find any hotels in Zuellig. Is there another location I can search for you?I couldn't find any hotels in Zuellig. Is there another location I can search for you?

Assistant: Here are some hotels in Zurich:

*   **Courtyard Zurich** (ID: 9) 

## Cleaning up

To avoid incurring ongoing charges to your Google Cloud account for the resources used in this tutorial, delete the resources you created. This section provides commands to delete the deployed Agent Engine and the associated Cloud Storage bucket.

In [ ]:
delete_agent_engine = True
delete_bucket = True

if delete_agent_engine:
    agent_engines = agent_engines.list(filter="display_name=reddit_assistant_agent")
    for agent_engine in agent_engines:
        agent_engine.delete(force=True)

if delete_bucket:
    !gsutil rm -r {BUCKET_URI}